# Preprocess

In [1]:
from bcd.container import BCDContainer
from bcd.preprocess.convert import ImageConverter, ImageConverterParams
from bcd.preprocess.filter import MeanFilter, MedianFilter, GaussianFilter, FilterParams


In [2]:
container = BCDContainer()
container.init_resources()
container.wire(
    packages=[
        "bcd.manage_data",
        "bcd.preprocess.convert",        
    ]
)

In [3]:
image_repo = container.repo.image()


In [4]:
def visualize(processor: str, n: int) -> None:
    condition = lambda df: df['processor'] == processor
    images = image_repo.get(condition=condition, n=n)
    for image in images:
        image.visualize()


## Stage 0: Convert Images

In [6]:
params = ImageConverterParams()
conv = ImageConverter(params=params, task_id="some task id")
conv.execute()
df = image_repo.get_meta()
# visualize(task='ImageConverter', n=4)
df.info()

  0%|          | 0/36 [00:00<?, ?it/s]

100%|██████████| 36/36 [00:31<00:00,  1.15it/s]


,id,case_id,mode,stage_id,stage,left_or_right_breast,image_view,abnormality_type,assessment,breast_density,...,range_pixel_values,mean_pixel_value,median_pixel_value,std_pixel_value,filepath,fileset,cancer,created,preprocessor,task_id
0,398064fc-c8ca-4cb5-8127-e9b346289595,Calcification-Test_P_01211_RIGHT_CC_2,dev,0,converted,RIGHT,CC,calcification,2,3,...,65535,"12,839.10",0,"17,342.60",data/image/1_dev/398064fc-c8ca-4cb5-8127-e9b34...,test,None,2023-10-24 03:45:28,ImageConverter,some task id
1,9f7e347b-a9f3-4182-a62f-0a8cd4cbe0bb,Calcification-Train_P_01864_LEFT_CC_3,dev,0,converted,LEFT,CC,calcification,2,2,...,65535,"13,501.30",172,"13,858.20",data/image/1_dev/9f7e347b-a9f3-4182-a62f-0a8cd...,train,None,2023-10-24 03:45:29,ImageConverter,some task id
2,0a9dd7ba-72a2-430f-b39b-f47efa86fa02,Calcification-Train_P_00471_LEFT_CC_1,dev,0,converted,LEFT,CC,calcification,2,3,...,65535,"19,976.80",15,"19,032.60",data/image/1_dev/0a9dd7ba-72a2-430f-b39b-f47ef...,train,None,2023-10-24 03:45:30,ImageConverter,some task id
3,c8306888-7db1-43b4-873d-76519da67d39,Calcification-Train_P_00847_LEFT_CC_2,dev,0,converted,LEFT,CC,calcification,4,4,...,65535,"8,773.86",0,"13,350.40",data/image/1_dev/c8306888-7db1-43b4-873d-76519...,train,None,2023-10-24 03:45:31,ImageConverter,some task id
4,437d9a93-9b54-4cd4-9dba-8b9ad1c7741d,Calcification-Train_P_01579_RIGHT_CC_1,dev,0,converted,RIGHT,CC,calcification,4,1,...,65535,"17,071.00",234,"17,075.30",data/image/1_dev/437d9a93-9b54-4cd4-9dba-8b9ad...,train,None,2023-10-24 03:45:31,ImageConverter,some task id
5,75485aed-ae91-45dc-aa30-56bc7d992a21,Calcification-Train_P_00655_RIGHT_CC_1,dev,0,converted,RIGHT,CC,calcification,4,2,...,65535,"9,117.53",0,"14,274.80",data/image/1_dev/75485aed-ae91-45dc-aa30-56bc7...,train,None,2023-10-24 03:45:33,ImageConverter,some task id
6,6e8c1884-5286-4b53-a8cd-2a23184b0190,Calcification-Train_P_00809_LEFT_CC_1,dev,0,converted,LEFT,CC,calcification,4,2,...,65535,"11,296.80",0,"18,561.70",data/image/1_dev/6e8c1884-5286-4b53-a8cd-2a231...,train,None,2023-10-24 03:45:33,ImageConverter,some task id
7,0e1d95c6-a8e6-4dff-9ad0-d972ad3eeb7b,Calcification-Test_P_00397_LEFT_CC_1,dev,0,converted,LEFT,CC,calcification,4,4,...,62569,"10,537.70",153,"12,421.80",data/image/1_dev/0e1d95c6-a8e6-4dff-9ad0-d972a...,test,None,2023-10-24 03:45:34,ImageConverter,some task id
8,06353668-33ba-4e18-9327-6ef445a480af,Calcification-Train_P_01296_RIGHT_CC_1,dev,0,converted,RIGHT,CC,calcification,5,2,...,65535,"9,880.16",0,"14,498.30",data/image/1_dev/06353668-33ba-4e18-9327-6ef44...,train,None,2023-10-24 03:45:35,ImageConverter,some task id
9,44826f27-c124-4abb-87fb-0d79ba3ebab0,Mass-Train_P_00859_LEFT_CC_1,dev,0,converted,LEFT,CC,mass,0,2,...,65535,"9,142.23",0,"12,950.50",data/image/1_dev/44826f27-c124-4abb-87fb-0d79b...,train,None,2023-10-24 03:45:36,ImageConverter,some task id


KeyError: 'task'

## Stage 1: Filter

### Stage 1.0: Mean Filter

In [ ]:
params = FilterParams()
filter = MeanFilter(params)
filter.execute()
visualize(task='MeanFilter', n=4)

### Stage 1.1: Median Filter

In [ ]:
params = FilterParams()
filter = MedianFilter(params)
filter.execute()
visualize(task='MedianFilter', n=4)

### Stage 1.1: Gaussian Filter

In [ ]:
params = FilterParams()
filter = GaussianFilter(params)
filter.execute()
visualize(task='GaussianFilter', n=4)

In [ ]:
import uuid
str(uuid.uuid4())